In [ ]:
import requests
import pandas as pd
from datetime import datetime


EVENTS_API_URL = "https://api.sympla.com.br/public/v4/events"
PARTICIPANTS_API_URL = "https://api.sympla.com.br/public/v3/events/{event_id}/participants"
API_TOKEN = ""
HEADERS = {"s_token": API_TOKEN}
PAGE_SIZE = 100
DATA_INICIAL = datetime(2020, 1, 01)
DATA_FINAL = datetime(2025, 6, 10)
OUTPUT_EVENTS_FILE = "eventos.csv"
OUTPUT_PARTICIPANTS_FILE = "participants_sem_dado_sensivel.csv"
OUTPUT_SENSITIVE_FILE = "participants_dado_sensivel.csv"
OUTPUT_COLUNAS_FINAIS = "colunas_finais.csv"

def fetch_events():
    """Busca todos os eventos da API."""
    all_events = []
    page = 1

    while True:
        params = {"page": page, "pageSize": PAGE_SIZE}
        try:
            response = requests.get(EVENTS_API_URL, headers=HEADERS, params=params)
            response.raise_for_status()
            data = response.json()
            events = data.get("data", [])

            if not events:
                break

            all_events.extend(events)
            page += 1
        except requests.exceptions.RequestException as e:
            print(f"Erro ao buscar eventos: {e}")
            break

    return all_events

def process_events(events):
    """Processa os eventos e retorna um DataFrame filtrado e formatado."""
    df_events = pd.DataFrame(events)
    df_events["start_date"] = pd.to_datetime(df_events["start_date"])

    df_events_filtrado = df_events[
        (df_events["start_date"] >= DATA_INICIAL) & (df_events["start_date"] <= DATA_FINAL)
    ]

    df_address = pd.json_normalize(df_events_filtrado["address"])
    df_address = df_address.rename(columns={"name": "local_evento"})

    df_events_filtrado = pd.concat([df_events_filtrado.reset_index(drop=True), df_address.reset_index(drop=True)], axis=1)
    df_events_filtrado = df_events_filtrado.drop("address", axis=1)

    df_events_filtrado = df_events_filtrado.rename(columns={
        "id": "id_evento",
        "name": "nome_evento",
        "city": "cidade_evento",
        "start_date": "data_evento",
        "state": "estado_evento",
        "country": "pais_evento"
    })

    df_events_final = df_events_filtrado[
        ["id_evento", "nome_evento", "local_evento", "data_evento", "cidade_evento", "estado_evento", "pais_evento"]
    ]

    return df_events_final

def extract_custom_form(row):
    """Extrai dados do campo 'custom_form'."""
    custom_data = {}
    for field in row.get("custom_form", []):
        custom_data[field["name"]] = field["value"]
    return pd.Series(custom_data)

def extract_checkin(row):
    """Extrai dados do campo 'checkin'."""
    if row.get("checkin"):
        checkin_data = row["checkin"][0]
        return pd.Series({
            "check_in": checkin_data["check_in"],
            "check_in_date": checkin_data["check_in_date"]
        })
    return pd.Series({"check_in": None, "check_in_date": None})

def get_participants_for_event(event_id):
    """Busca participantes de um evento específico."""
    url = PARTICIPANTS_API_URL.format(event_id=event_id)
    params = {"page": 1, "pageSize": PAGE_SIZE}
    participants = []

    while True:
        try:
            response = requests.get(url, headers=HEADERS, params=params)
            response.raise_for_status()
            data = response.json()
            event_participants = data.get("data", [])

            if not event_participants:
                break

            for participant in event_participants:
                participant["event_id"] = event_id

            participants.extend(event_participants)
            params["page"] += 1
        except requests.exceptions.RequestException as e:
            print(f"Erro ao buscar participantes do evento {event_id}: {e}")
            break

    return participants

def concatenar_colunas(row, colunas_originais, nova_coluna):
    """Seleciona apenas a primeira resposta válida entre várias colunas."""
    for col in colunas_originais:
        if col in row and pd.notna(row[col]):
            valor = str(row[col]).strip()
            if valor:
                row[nova_coluna] = valor
                break
    else:
        row[nova_coluna] = None

    return row.drop(colunas_originais, errors="ignore")


def process_participants(df_participants):
    """Processa e limpa os dados dos participantes."""

    df_custom = df_participants.apply(extract_custom_form, axis=1)
    df_checkin = df_participants.apply(extract_checkin, axis=1)

    # Combina os dados
    df_participants = pd.concat([df_participants, df_custom, df_checkin], axis=1)
    df_participants.drop(columns=["custom_form", "checkin"], inplace=True)

    # Lista de colunas para processar (incluindo as novas colunas de nome completo)
    colunas_para_processar = [

    # Nome completo
    ([
        "Confirme seu nome completo, por favor", "Confirme seu nome completo, por favor ",
        "Confirme seu nome completo, por favor!", "Confirme seu nome completo, por favor! ",
        "Por gentileza, coloque seu nome completo:"
    ], "nome_completo"),

    # RA
    (["RA", "Ra", "Ra "], "registro_ra"),

    # CPF
    (["CPF", "CPF "], "cpf"),

    # CEP
    (["CEP", "Cep"], "cep"),

    # Data de nascimento
    ([
        "DATA DE NASCIMENTO ", "Data De Nascimento", "Data de Nascimento",
        "Data de Nascimento ", "Data de nascimento", "Data de nascimento "
    ], "data_nascimento"),

    # Certificado
    ([
        "Deseja Receber certificado", "Deseja receber certificado de horas?",
        "Deseja receber certificado de horas?  (Para que seja  emitido precisamos que seu nome e sobrenome  estejam preenchidos corretamente)",
        "Deseja receber certificado de horas? (Para que seja emitido precisamos que seu nome e sobrenome estejam preenchidos corretamente)",
        "Deseja receber certificado?",
        "Deseja receber certificado? ",
        "Deseja receber certificado? (Para que seja emitido precisamos que seu nome e sobrenome estejam preenchidos corretamente)",
        "Deseja receber certificado?  (Para que seja  emitido precisamos que seu nome e sobrenome  estejam preenchidos corretamente)",
        "Deseja receber certificado? (Para que seja emitido precisamos que seu nome e sobrenome estejam preenchidos corretamente) ",
        "Deseja receber centificado de horas?",
        "Deseja receber certificado? (Para que seja  emitido precisamos que seu nome e sobrenome  estejam preenchidos corretamente)",
        "Deseja receber centificado?"
    ], "certificado"),

    # Etnia
    (["ETNIA", "Etnia", "Etinia", "Etnia ","Você se considera uma pessoa"], "etnia"),

    # Gênero
    (["GÊNERO", "Gênero", "Gênero ", "Qual a sua identidade de gênero?"], "genero"),

    # Como ficou sabendo
    ([
        "COMO FICOU SABENDO DA AÇÃO?", "Como ficou sabendo da ação?",
        "Como ficou sabendo da ação? ", "Como ficou sabendo do evento?",
        "Como ficou sabendo do evento? ", "Porque se interessou pelo projeto? "
    ], "como_ficou_sabendo"),

    # Primeira ação
    ([
        "É a sua primeira ação como voluntário?", "É a sua primeira ação?",
        "É a sua primeira ação na ação?", "É a sua primeira ação na ação? ",
        "É a sua primeira vez como voluntário na ação?", "É a sua primeira vez na ação?",
        "É a sua primeira vez na ação? ", "É sua primeira ação como voluntário?",
        "É sua primeira vez como voluntário na ação?", "É a sua primeira vez participando do Coração em Plantão?"
    ], "primeira_acao"),

    # Política de privacidade
    ([
        "Política de Privacidade (Lei Geral de Proteção de Dados)",
        "Política de Privacidade Inscrições -Somando Mais Ações - Bem Da Madrugada",
        "Política de Privacidade Inscrições -Somando Mais Ações - Bem Da Madrugada ",
        "Política de Privacidade Inscrições e Termo de Uso de Equipamento de Segurança - Somando Mais  Ações - Bem Da Madrugada",
        "Política de Privacidade Inscrições e Termo de Uso de Equipamento de Segurança - Somando Mais Ações - Bem Da Madrugada",
        "Política de Privacidade Inscrições e Termo de Uso de Equipamento de Segurança - Somando Mais Ações - Bem Da Madrugada ",
        "Política de Privacidade Inscrições e Termo de Uso de Equipamento de Segurança - Somando Mais  Ações - Bem Da Madrugada "
    ], "politica_privacidade"),

    # Qual semestre
    ([
        "Em qual semestre está?", "Em qual semestre você está?",
        "Em qual semestre você está? ", "Em qual semestre você está? *",
    ], "qual_semestre"),

    # WhatsApp
    ([
        "É WHATSAPP?", "É WhatsApp", "É WhatsApp?", "É whatsapp?",
        "é WhatsApp?", "é whatsapp?", "É whatsapp"
    ], "e_whatsapp"),

    # Profissão
    ([
        "Profissão", "PROFISSÃO", "PROFISSÃO ",
        "Profissão (ex.: Estudante de Medicina/Matemática)",
        "Profissão (ex.: Estudante de Medicina/Matemática)?",
        "Qual a sua área de atuação?", "Em qual área você atua?",
        "Você está estudando no momento? Se sim, qual o seu curso?"
    ], "profissao"),

    # Toca instrumento
    ([
        "Toca Algum instrumento", "Toca algum instrumento?", "Toca algum instrumento? ",
        "Toca algum instrumento musical?", "Você toca algum instrumento musical?",
        "Você toca algum instrumento?", "Você toca algum instrumento musical?"
    ], "toca_instrumento"),

    # Qual instrumento
    ([
        "Qual instrumento?", "Se sim, qual instrumento?",
        "Se sim, qual nstrumento?", "Se sim, qual?"
    ], "qual_instrumento"),

    # O que motivou a participar
    ([
        "O que motivou a participar?", "O que motivou a participar? ",
        "O que o motivou a participar?", "O QUE TE MOTIVOU A PARTICIPAR?",
        "o que motivou a participar"
    ], "o_que_motivou"),

    # Quantas ações já participou
    ([
        "Se essa não é a sua primeira ação, em quantas já foi?",
        "Caso não seja sua primeira vez, quantas vezes você já participou?"
    ], "quantas_acoes"),

    # Endereço
    (["Endereço","Em qual bairro você mora?"], "endereco"),

    # Telefone
    (["Telefone"], "telefone"),

]

    # Processa as colunas
    for colunas_originais, nova_coluna in colunas_para_processar:
        df_participants = df_participants.apply(concatenar_colunas, axis=1, args=(colunas_originais, nova_coluna))

    colunas_para_remover = [
        "Possui carro para nos ajudar a distribuir nas rotas (norte, sul, leste e oeste)?",
        "Deseja adquirir a camiseta do projeto?",
        "Caso tenha carro, em qual rota pode ajudar a distribuir?",
        "ATENÇÃO! Em caso de solicitação de certificado, verifique se o os dados (nome e RA) estão completos e corretos para evitar atrasos no envio",

    ]
    df_participants.drop(columns=[col for col in colunas_para_remover if col in df_participants.columns], inplace=True)

    # Limpeza e formatação
    df_participants["data_nascimento"] = pd.to_datetime(df_participants["data_nascimento"], format="%d/%m/%Y", errors="coerce")
    df_participants["email"] = df_participants["email"].str.upper()

    return df_participants

def main():
    # Busca todos os eventos
    events = fetch_events()
    if not events:
        print("Nenhum evento encontrado.")
        return

    # Processa os eventos
    df_eventos = process_events(events)
    df_eventos.to_csv(OUTPUT_EVENTS_FILE, index=False)
    print(f"Eventos salvos em {OUTPUT_EVENTS_FILE}")

    # Busca participantes para todos os eventos
    todos_participantes = []
    for id_evento in df_eventos["id_evento"]:
        participantes = get_participants_for_event(id_evento)
        todos_participantes.extend(participantes)

    # Cria e trata o DataFrame de participantes
    df_participantes = pd.DataFrame(todos_participantes)
    df_participantes = process_participants(df_participantes)

    # Renomeia colunas e mescla com eventos
    df_participantes = df_participantes.rename(columns={"event_id": "id_evento"})
    df_completo = pd.merge(df_participantes, df_eventos, on="id_evento", how="left")

    # Renomeia colunas para o formato final
    df_completo = df_completo.rename(columns={
        "id": "id_participante_sympla",
        "ticket_name": "tipo_ingresso",
        "first_name": "nome",
        "last_name": "sobrenome",
        "Telefone": "telefone",
        "check_in": "check_in",
        "check_in_date": "data_checkin"
    })

    # Define as colunas sem dados sensíveis
    colunas_sem_dado_sensivel = [
        "id_participante_sympla", "id_evento", "nome_evento", "data_evento", "tipo_ingresso", "pais_evento", "estado_evento", "cidade_evento",
        "nome", "sobrenome", "email", "cpf", "data_nascimento", "telefone", "e_whatsapp", "endereco", "cep", "profissao",
        "como_ficou_sabendo", "o_que_motivou", "primeira_acao", "quantas_acoes", "certificado", "registro_ra", "qual_semestre", "toca_instrumento",
        "qual_instrumento", "politica_privacidade", "check_in", "data_checkin"
    ]

    # Cria o DataFrame sem dados sensíveis
    df_final_sem_sensiveis = df_completo[colunas_sem_dado_sensivel]

    # Cria o DataFrame apenas com dados sensíveis
    colunas_sensiveis = ["id_evento", "nome_evento", "cidade_evento", "genero", "etnia"]
    df_dados_sensiveis = df_completo[colunas_sensiveis]

    # Exporta os arquivos
    df_final_sem_sensiveis.to_csv(OUTPUT_PARTICIPANTS_FILE, index=False, encoding="utf-8-sig")
    df_dados_sensiveis.to_csv(OUTPUT_SENSITIVE_FILE, index=False, encoding="utf-8-sig")
    df_completo.to_csv(OUTPUT_COLUNAS_FINAIS, index=False, encoding="utf-8-sig")

    print(f"Participantes salvos em {OUTPUT_PARTICIPANTS_FILE}")
    print(f"Dados sensíveis salvos em {OUTPUT_SENSITIVE_FILE}")
    print(f"Colunas finais salvas em {OUTPUT_COLUNAS_FINAIS}")



if __name__ == "__main__":
    main()